In [ ]:
# pip install openai neo4j python-dotenv

In [8]:
# from openai import OpenAI
import openai
import json
import urllib
import pandas as pd
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed  # Added missing imports
import concurrent.futures
import re
from tqdm import tqdm
import nltk
nltk.download('punkt')  # Ensure the punkt tokenizer is downloaded



ImportError: cannot import name 'is_union' from 'pydantic.typing' (/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/pydantic/typing.cpython-39-darwin.so)

In [3]:
load_dotenv()


NameError: name 'load_dotenv' is not defined

In [8]:
# from openai import OpenAI
import openai
import json
import urllib
import pandas as pd
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed  # Added missing imports
import concurrent.futures
import re
from tqdm import tqdm
import nltk
nltk.download('punkt')  # Ensure the punkt tokenizer is downloaded


# load_dotenv()
# openai.api_key = os.getenv("OPENAI_KEY")
# # openai.api_version = os.getenv("api_version")
# openai.api_type = "openai"


openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-4"


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davidoluyalegbenga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
uri = os.getenv("NEO4J_CONNECTION_URL")
username = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")



In [10]:
driver = GraphDatabase.driver(uri, auth=(username, password))


In [11]:
def execute_cypher_query(cypher_query):
    driver = GraphDatabase.driver(uri, auth=(username, password))
    
    try:
        with driver.session() as session:
            session.run(cypher_query)
    
    except Exception as e:
        return 0
        # print(f"An error occurred while executing the Cypher query: {e}")
        # Handle the error here (e.g., log the error, raise an exception, etc.)
    
    finally:
        driver.close()  # Close the Neo4j driver after execution (important)


In [12]:
# Function to extract sentences from text content
def extract_sentences(text):
    return text.split('。')  # Split text by full stop (assuming full stop is '。' in Chinese)

# def extract_sentences2(text):
#     # Splitting by sentences
#     sentences = re.split(r'(?<=[.!?]) +', text)
#     # Splitting by paragraphs (assuming double line breaks separate paragraphs)
#     paragraphs = text.split('\n\n')  # or text.split('\n\n\n') if triple line breaks
#     return sentences, paragraphs

# Function to extract text within quotes from a string
def extract_quoted_text(text):
    return re.findall(r'"([^"]*)"', text)


def custom_sentence_tokenizer(text, max_words=50):
    words = text.split()
    sentences = []
    current_sentence = []

    for word in words:
        current_sentence.append(word)
        if len(current_sentence) >= max_words and (word.endswith('.') or word.endswith('!') or word.endswith('?')):
            sentences.append(' '.join(current_sentence))
            current_sentence = []

    if current_sentence:
        sentences.append(' '.join(current_sentence))

    return sentences


def extract_sentences2(text):
    # Splitting by sentences
    sentences = nltk.sent_tokenize(text)
    return sentences

In [13]:
def process_sentence(sentence):
    try:
        prompt = f"""
        Given the  text content "{sentence}" generate Neo4j Cypher queries to create a knowledge graph based on the above. Only return the Cypher only. No explanations etc.
        """            
        response = openai.chat.completions.create(
                    model=openai_deployment,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt},
                    ]
                )

        # Extract the category from the response
        cypherCode = response.choices[0].message.content
        print(cypherCode)
        return cypherCode

    except Exception as e:
        print(f"OPEN AI An error occurred: {e}")
        return None  # Return None or handle the error as needed


In [14]:
def process_files_concurrently(file_paths):
    openai_responses = []
    total_files = len(file_paths)

    with ThreadPoolExecutor(max_workers=5000) as executor_files:
        futures_files = {executor_files.submit(process_text_file, file_path): file_path for file_path in file_paths}
        with tqdm(total=total_files, desc='Processing Files') as pbar:
            for future_file in tqdm(as_completed(futures_files), total=total_files, desc='Files'):
                file_path = futures_files[future_file]
                data = future_file.result()
                if data:
                    openai_responses.append(data)
                pbar.update(1)  # Update progress bar for each file processed
                pbar.set_description(f"Processed {pbar.n}/{total_files} files")
    return openai_responses

In [13]:
def execute_queries_concurrently(queries):
    execute_cypher_query(queries)

In [15]:
def process_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
        extracted_texts = file_content #extract_quoted_text(file_content)
        if extracted_texts:
            extracted_text = extracted_texts  # Considering the first quoted text
            
            sentences = custom_sentence_tokenizer(extracted_text)
            print(f"extracted_textssentences=== {len(sentences)}")

            with ThreadPoolExecutor(max_workers=3000) as executor_sentences:
                futures_sentences = {executor_sentences.submit(process_sentence, sentence): sentence for sentence in sentences}
                processed_cypher_queries = []
                for future_sentence in tqdm(as_completed(futures_sentences), total=len(sentences), desc='Sentences'):
                    query = future_sentence.result()
                    if query:
                        processed_cypher_queries.append(query)
                        execute_queries_concurrently(query)
                
            return processed_cypher_queries
        else:
            return None


In [ ]:
# Path to the folder containing .txt files downloaded_text
folder_path = 'Data/cleaned_text_manually'

# Array to store OpenAI responses
openai_responses = []

# List to hold file paths
file_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.txt')]

process_files_concurrently(file_paths)

### Test output


In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.chat_models import ChatOpenAI

graph = Neo4jGraph(
    url=uri,
    username=username,
    password=password
)
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    verbose=True,
)
response = chain("what benefits does UNDP offer to Afghanistan") #test the chain
print(f"Final answer: {response['result']}")
